<a href="https://colab.research.google.com/github/Paradoxxs/Paradoxxs.github.io/blob/main/pcap_network_anlysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using NFStream to convert the pcap file to dataframe, and there after perform analysis.

In [ ]:
pip install nfstream

In [2]:
import nfstream
from nfstream import NFStreamer, NFPlugin
import pandas as pd
import numpy
from sklearn.ensemble import RandomForestClassifier

In [ ]:
data_df = NFStreamer(source="file.pcap").to_pandas()
data_df.head()

In [40]:
import math
import pandas as pd
import numpy as np

In [41]:
f_timestamp = 'bidirectional_first_seen_ms'
f_src_ip = 'src_ip'
f_dst_ip = 'dst_ip'
f_dst_host = 'requested_server_name'
f_dst_port = 'dst_port'
f_sent_bytes = 'src2dst_bytes'


columns_to_filter = [f_timestamp, f_src_ip, f_dst_ip, f_dst_host, f_dst_port, f_sent_bytes]
columns_to_groupby = [f_src_ip, f_dst_ip, f_dst_port] #Group the connect together that are the same. 
# columns to display after the analysis
columns_to_display = ['Score','tsScore','dsScore','conn_count',f_src_ip,f_dst_ip,f_dst_host, f_dst_port,f_sent_bytes,'deltas']

In [42]:
data_df = data_df.loc[:,columns_to_filter]
data_df[f_timestamp] = pd.to_datetime(data_df[f_timestamp], unit='ms')  #Converting ms to datetime
data_df = data_df.groupby(columns_to_groupby).agg(list)
data_df.reset_index(inplace=True)
data_df.head()

,src_ip,dst_ip,dst_port,bidirectional_first_seen_ms,requested_server_name,src2dst_bytes
0,0.0.0.0,255.255.255.255,67,[2022-01-07 16:04:09.195000],[desktop-gxmyno2],[379]
1,192.168.1.1,192.168.1.216,68,[2022-01-07 16:04:09.195000],[nan],[342]
2,192.168.1.2,192.168.1.216,137,[2022-01-07 16:16:09.958000],[spoonwatch-dc],[104]
3,192.168.1.2,192.168.1.216,49719,"[2022-01-07 16:11:30.497000, 2022-01-07 16:13:...","[nan, nan]","[55, 564]"
4,192.168.1.216,104.212.67.47,443,[2022-01-07 16:07:09.863000],[pti.store.microsoft.com],[2866]


In [43]:
data_df.shape

(38, 6)

In [44]:
# create a new column 'conn_count', and for each row in the 'timestamp' column, apply a function and assign the returned value to the 'conn_count' column
data_df['ConnectionCount'] = data_df[f_timestamp].apply(lambda x: len(x))
data_df.head()

,src_ip,dst_ip,dst_port,bidirectional_first_seen_ms,requested_server_name,src2dst_bytes,ConnectionCount
0,0.0.0.0,255.255.255.255,67,[2022-01-07 16:04:09.195000],[desktop-gxmyno2],[379],1
1,192.168.1.1,192.168.1.216,68,[2022-01-07 16:04:09.195000],[nan],[342],1
2,192.168.1.2,192.168.1.216,137,[2022-01-07 16:16:09.958000],[spoonwatch-dc],[104],1
3,192.168.1.2,192.168.1.216,49719,"[2022-01-07 16:11:30.497000, 2022-01-07 16:13:...","[nan, nan]","[55, 564]",2
4,192.168.1.216,104.212.67.47,443,[2022-01-07 16:07:09.863000],[pti.store.microsoft.com],[2866],1


In [45]:
data_df = data_df.loc[data_df['ConnectionCount'] > 10] #Remove all connection with less then 10 connections, it was choosen because of the small data sample I used, The goal is to reduce the amount of data that need to be processed
data_df.head()

,src_ip,dst_ip,dst_port,bidirectional_first_seen_ms,requested_server_name,src2dst_bytes,ConnectionCount
7,192.168.1.216,192.168.1.2,53,"[2022-01-07 16:04:09.662000, 2022-01-07 16:04:...","[_ldap._tcp.dc._msdcs.spoonwatch.net, spoonwat...","[95, 88, 107, 107, 76, 126, 130, 79, 76, 131, ...",44
8,192.168.1.216,192.168.1.2,88,"[2022-01-07 16:04:10.951000, 2022-01-07 16:04:...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[472, 472, 606, 606, 418, 607, 2206, 2190, 198...",24
12,192.168.1.216,192.168.1.2,389,"[2022-01-07 16:04:09.705000, 2022-01-07 16:04:...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[264, 2894, 264, 265, 308, 264, 311, 265, 264,...",31


In [46]:
data_df[f_timestamp] = data_df[f_timestamp].apply(lambda x: sorted(x))
data_df.head()

,src_ip,dst_ip,dst_port,bidirectional_first_seen_ms,requested_server_name,src2dst_bytes,ConnectionCount
7,192.168.1.216,192.168.1.2,53,"[2022-01-07 16:04:09.662000, 2022-01-07 16:04:...","[_ldap._tcp.dc._msdcs.spoonwatch.net, spoonwat...","[95, 88, 107, 107, 76, 126, 130, 79, 76, 131, ...",44
8,192.168.1.216,192.168.1.2,88,"[2022-01-07 16:04:10.951000, 2022-01-07 16:04:...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[472, 472, 606, 606, 418, 607, 2206, 2190, 198...",24
12,192.168.1.216,192.168.1.2,389,"[2022-01-07 16:04:09.705000, 2022-01-07 16:04:...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[264, 2894, 264, 265, 308, 264, 311, 265, 264,...",31


In [47]:
# Convert list into a Series object, get time delta, convert the result back into a list and assign it to the 'deltas' column
data_df['delta_time'] = data_df[f_timestamp].apply(lambda x: pd.Series(x).diff().dt.seconds.dropna().tolist())
data_df.head()

,src_ip,dst_ip,dst_port,bidirectional_first_seen_ms,requested_server_name,src2dst_bytes,ConnectionCount,delta_time
7,192.168.1.216,192.168.1.2,53,"[2022-01-07 16:04:09.662000, 2022-01-07 16:04:...","[_ldap._tcp.dc._msdcs.spoonwatch.net, spoonwat...","[95, 88, 107, 107, 76, 126, 130, 79, 76, 131, ...",44,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,192.168.1.216,192.168.1.2,88,"[2022-01-07 16:04:10.951000, 2022-01-07 16:04:...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[472, 472, 606, 606, 418, 607, 2206, 2190, 198...",24,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
12,192.168.1.216,192.168.1.2,389,"[2022-01-07 16:04:09.705000, 2022-01-07 16:04:...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[264, 2894, 264, 265, 308, 264, 311, 265, 264,...",31,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [48]:
data_df['tsLow'] = data_df['delta_time'].apply(lambda x: np.percentile(np.array(x),25))
data_df['tsMid'] = data_df['delta_time'].apply(lambda x: np.percentile(np.array(x), 50))
data_df['tsHigh'] = data_df['delta_time'].apply(lambda x: np.percentile(np.array(x), 75))

data_df['tsBowleyNum'] = data_df['tsLow'] + data_df['tsHigh'] - 2 * data_df['tsMid']
data_df['tsBowleyDen'] = data_df['tsHigh'] - data_df['tsLow']

# tsSkew should equal zero if the denominator equals zero
# bowley skew is unreliable if Q2 = Q1 or Q2 = Q3
data_df['tsSkew'] = data_df[['tsLow', 'tsMid', 'tsHigh', 'tsBowleyNum','tsBowleyDen']].apply(
    lambda x: x['tsBowleyNum'] / x['tsBowleyDen'] if x['tsBowleyDen'] !=0 and x['tsMid'] != x['tsLow'] and x['tsMid'] != x['tsHigh'] !=0 else 0.0, axis=1
    )
data_df['tsMadm'] = data_df['delta_time'].apply(lambda x: np.median(np.absolute(np.array(x) - np.median(np.array(x)))))
data_df['tsConnDiv'] = data_df[f_timestamp].apply(lambda x: (x[-1].to_pydatetime() - x[0].to_pydatetime()).seconds / 90)

# Time delta score calculation
data_df['tsConnCountScore'] = data_df.apply(lambda x: 0.0 if x['tsConnDiv'] == 0  else x['ConnectionCount'] / x['tsConnDiv'] if x['ConnectionCount'] / x['tsConnDiv'] < 1.0 else 1.0 , axis=1)
data_df['tsSkewScore'] = 1.0 - abs(data_df['tsSkew'])
data_df['tsMadmScore'] = data_df['tsMadm'].apply(lambda x: 0 if 1.0 - (x / 30.0) < 0 else 1.0 - (x / 30.0))
data_df['tsScore'] = (((data_df['tsSkewScore'] + data_df['tsMadmScore'] + data_df['tsConnCountScore']) / 3.0) * 1000) / 1000


In [49]:
data_df['dsLow'] = data_df[f_sent_bytes].apply(lambda x: np.percentile(np.array(x), 25))
data_df['dsMid'] = data_df[f_sent_bytes].apply(lambda x: np.percentile(np.array(x), 50))
data_df['dsHigh'] = data_df[f_sent_bytes].apply(lambda x: np.percentile(np.array(x), 75))
data_df['dsBowleyNum'] = data_df['dsLow'] + data_df['dsHigh'] - 2 * data_df['dsMid']
data_df['dsBowleyDen'] = data_df['dsHigh'] - data_df['dsLow']


# dsSkew should equal zero if the denominator equals zero
# bowley skew is unreliable if Q2 = Q1 or Q2 = Q3
data_df['dsSkew'] = data_df[['dsLow','dsMid','dsHigh','dsBowleyNum','dsBowleyDen']].apply(
    lambda x: x['dsBowleyNum'] / x['dsBowleyDen'] if x['dsBowleyDen'] != 0 and x['dsMid'] != x['dsLow'] and x['dsMid'] != x['dsHigh'] else 0.0, axis=1
    )
data_df['dsMadm'] = data_df[f_sent_bytes].apply(lambda x: np.median(np.absolute(np.array(x) - np.median(np.array(x)))))


# Data size score calculation of sent bytes
data_df['dsSkewScore'] = 1.0 - abs(data_df['dsSkew'])
data_df['dsMadmScore'] = data_df['dsMadm'].apply(lambda x: 0 if x/ 128.0 < 0 else x/ 128.0)
data_df['dsSmallnessScore'] = data_df['dsMid'].apply(lambda x: 0 if 1.0 - x / 8192.0 < 0 else 1.0 - x / 8192.0)
data_df['dsScore'] = (((data_df['dsSkewScore'] + data_df['dsMadmScore'] + data_df['dsSmallnessScore']) / 3.0) * 1000) / 1000

In [51]:
# Overal Score calculation
data_df['Score'] = (data_df['dsScore'] + data_df['tsScore']) / 2

data_df.sort_values(by= 'Score', ascending=False, inplace=True, ignore_index=True)
data_df.head(30)

,src_ip,dst_ip,dst_port,bidirectional_first_seen_ms,requested_server_name,src2dst_bytes,ConnectionCount,delta_time,tsLow,tsMid,tsHigh,tsBowleyNum,tsBowleyDen,tsSkew,tsMadm,tsConnDiv,tsConnCountScore,tsSkewScore,tsMadmScore,tsScore,dsLow,dsMid,dsHigh,dsBowleyNum,dsBowleyDen,dsSkew,dsMadm,dsSkewScore,dsMadmScore,dsSmallnessScore,dsScore,Score
0,192.168.1.216,192.168.1.2,88,"[2022-01-07 16:04:10.951000, 2022-01-07 16:04:...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[472, 472, 606, 606, 418, 607, 2206, 2190, 198...",24,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.455556,1.0,1.0,1.0,1.0,602.75,1981.0,2154.0,-1205.25,1551.25,-0.776954,225.0,0.223046,1.757812,0.758179,0.913012,0.956506
1,192.168.1.216,192.168.1.2,53,"[2022-01-07 16:04:09.662000, 2022-01-07 16:04:...","[_ldap._tcp.dc._msdcs.spoonwatch.net, spoonwat...","[95, 88, 107, 107, 76, 126, 130, 79, 76, 131, ...",44,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,7.0,7.0,7.0,0.0,0.0,8.011111,1.0,1.0,1.0,1.0,79.00,89.0,94.0,-5.00,15.00,-0.333333,10.0,0.666667,0.078125,0.989136,0.577976,0.788988
2,192.168.1.216,192.168.1.2,389,"[2022-01-07 16:04:09.705000, 2022-01-07 16:04:...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[264, 2894, 264, 265, 308, 264, 311, 265, 264,...",31,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.133333,1.0,1.0,1.0,1.0,264.00,265.0,2915.5,2649.50,2651.50,0.999246,46.0,0.000754,0.359375,0.967651,0.442594,0.721297
